In [10]:
# FASE 5: DESPLIEGUE EN HUGGING FACE SPACES (ESTRUCTURA DE TU CLASE + CORRECCIONES)
!pip install --no-cache-dir ultralytics easyocr gradio huggingface_hub --quiet

import os
import cv2
import easyocr
import numpy as np
import gradio as gr
from ultralytics import YOLO
from huggingface_hub import login, HfApi

# 1. LOGIN CON TU TOKEN (como en tu notebook de clase)
login("asdsfD")

api = HfApi()

# 2. TU SPACE EXISTENTE (el mismo que usaste)
space_repo = "Juaano/ImagenClassificationSpace"

print(f"Usando Space: https://huggingface.co/spaces/{space_repo}")

# 3. SUBIR MODELO (.pt) – asegúrate de tenerlo subido en Colab
model_local_path = "best_plate_yolo11m.pt"  # ← sube tu archivo a Colab

api.upload_file(
    path_or_fileobj=model_local_path,
    path_in_repo="best_plate_yolo11m.pt",
    repo_id=space_repo,
    repo_type="space"
)
print("Modelo .pt subido")

# 4. app.py (tu función de inferencia + OCR)
app_code = '''
import cv2
import easyocr
import numpy as np
from ultralytics import YOLO
import gradio as gr

model = YOLO("best_plate_yolo11m.pt")
reader = easyocr.Reader(['es'], gpu=False)

def process_image(img):
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    results = model(img_cv)[0]

    texto_placa = "NO SE DETECTÓ PLACA"

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()

        placa = img_cv[y1:y2, x1:x2]
        ocr = reader.readtext(placa, detail=0)
        texto = "".join(ocr).upper().replace(" ", "").replace("-", "") if ocr else "NO_LEIDO"

        if conf > 0.3:
            texto_placa = f"{texto} (conf: {conf:.2f})"

        cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0, 255, 0), 4)

    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    return img_rgb, texto_placa

gr.Interface(
    fn=process_image,
    inputs=gr.Image(label="Sube foto o usa webcam", type="pil"),
    outputs=[
        gr.Image(label="Resultado con detección"),
        gr.Textbox(label="Placa leída")
    ],
    title="PlateFlow: Detección y Lectura de Placas Vehiculares",
    description="Proyecto Final PDI – Juaano28 – 2025",
    allow_flagging="never",
    live=True
).launch()
'''

# 5. requirements.txt CORREGIDO (headless para evitar libGL)
requirements = """
ultralytics
easyocr
gradio
opencv-python-headless
"""

# 6. GUARDAR Y SUBIR
os.makedirs("app", exist_ok=True)

with open("app/app.py", "w") as f:
    f.write(app_code)

with open("app/requirements.txt", "w") as f:
    f.write(requirements)

# Subir archivos
api.upload_file(path_or_fileobj="app/app.py", path_in_repo="app.py", repo_id=space_repo, repo_type="space")
api.upload_file(path_or_fileobj="app/requirements.txt", path_in_repo="requirements.txt", repo_id=space_repo, repo_type="space")

print("\n¡Todo subido correctamente!")
print(f"Link del Space: https://huggingface.co/spaces/{space_repo}")
print("Ve al Space y haz 'Restart Space' (arriba derecha).")
print("En 1-3 minutos la app arranca sin errores.")

Usando Space: https://huggingface.co/spaces/Juaano/ImagenClassificationSpace


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  best_plate_yolo11m.pt       :  83%|########2 | 33.5MB / 40.5MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Modelo .pt subido


No files have been modified since last commit. Skipping to prevent empty commit.



¡Todo subido correctamente!
Link del Space: https://huggingface.co/spaces/Juaano/ImagenClassificationSpace
Ve al Space y haz 'Restart Space' (arriba derecha).
En 1-3 minutos la app arranca sin errores.


In [12]:
# SOLUCIÓN DEFINITIVA: AGREGAR packages.txt CON libgl1
from huggingface_hub import HfApi, login

login("Access")

space_repo = "Juaano/ImagenClassificationSpace"

api = HfApi()

# Crear packages.txt con libgl1
with open("packages.txt", "w") as f:
    f.write("libgl1\nlibglib2.0-0\nffmpeg\n")  # extras por si acaso

# Subir packages.txt
api.upload_file(
    path_or_fileobj="packages.txt",
    path_in_repo="packages.txt",
    repo_id=space_repo,
    repo_type="space"
)

print("packages.txt subido con libgl1")
print("Ve a tu Space: https://huggingface.co/spaces/Juaano/ImagenClassificationSpace")
print("Haz 'Restart Space'.")
print("En 1-3 minutos la app arranca sin error de libGL.")

packages.txt subido con libgl1
Ve a tu Space: https://huggingface.co/spaces/Juaano/ImagenClassificationSpace
Haz 'Restart Space'.
En 1-3 minutos la app arranca sin error de libGL.
